In [266]:
import re
import os
import pandas as pd
import random
import json

In [267]:
suffix = ""
FILE_DISFLUENT_BASE = 'real_disf'+suffix+'.txt'

#FILE_FLUENT_BASE = './data_benchmark/real_fluent.txt'

In [268]:
DISFLUENT_EXAMPLES = [u[:-1] for u in open(FILE_DISFLUENT_BASE,'r', encoding = 'utf-8').readlines()]
#FLUENT_EXAMPLES = [u[:-1] for u in open(FILE_FLUENT_BASE,'r').readlines()]

#DISFLUENT_DF_LENGTHS = pd.Series([len(u.split()) for u in DISFLUENT_EXAMPLES]).plot(kind='box')

In [269]:
DISFLUENT_DF = pd.DataFrame(DISFLUENT_EXAMPLES,columns=['text'])
#DISFLUENT_DF['disfluent'] =True
#DISFLUENT_DF['text'] = DISFLUENT_DF['text'].str.replace('#', ',')
#FLUENT_DF = pd.DataFrame(FLUENT_EXAMPLES,columns=['text'])
#FLUENT_DF['disfluent'] =False


In [270]:
#ALL_DF = pd.concat([DISFLUENT_DF,FLUENT_DF],axis=0) 
#ALL_DF.to_csv('disfluency_fr.csv',index=False)

In [271]:
DISFLUENT_DF

,text
0,c' était un bar avec euh un un écran ou #
1,je crois j' étais euh je me rappelle plus si j...
2,on a appelé euh des # * les parents d' amis #
3,mais jetés comme des euh # comme des miséreux ...
4,on en pouvait plus de euh d' attendre *
...,...
195,le fait d' avoir la ville et puis le fait d' a...
196,ah pff ah moi j' ai pas envie de repartir euh ...
197,et qu' ils aient pas euh # euh # un terrain eu...
198,mais euh # moi j' avais un j' avais un voisin ...


In [272]:
if suffix == "_en":
    DISF_ITEMS = ['um','uh']
    with open('real_disf_en.txt', 'r', encoding = 'utf-8') as f:
        FAUX_DISF = [x for x in f.read().split() if x not in ['um','uh'] + ['euh'] + ['#', '*', ',']]
else:
    DISF_ITEMS = ['euh']
    with open('real_disf.txt', 'r', encoding = 'utf-8') as f:
        FAUX_DISF = [x for x in f.read().split() if x not in ['um','uh'] + ['euh'] + ['#', '*', ',']]

   
# MODIFYING GENUINE DISFLUENT EXAMPLES
def move_disfluencies(row):
    utt_as_list = row['text'].split()
    filtered_utt = [t for t in utt_as_list if t not in DISF_ITEMS]
    items_to_move = [t for t in utt_as_list if t in DISF_ITEMS]
    new_utt = utt_as_list
    while new_utt == utt_as_list:
        new_utt = filtered_utt
        for item in items_to_move:
            insert_location = random.randrange(2, len(new_utt)-2)
            new_utt.insert(insert_location,item) 
    return ' '.join(new_utt)

def remove_disfluencies(row):
    utt_as_list = row['text'].split()
    return ' '.join([t for t in utt_as_list if t not in DISF_ITEMS])

def replace_disfluencies(row):
    utt_as_list = row['text'].split()
    targets = [x for x in utt_as_list if x not in DISF_ITEMS + ['#', '*', ',']]
    for disf_item in DISF_ITEMS:
        utt_as_list = [re.sub("^"+disf_item+"$", random.choice(FAUX_DISF), x) for x in utt_as_list]
    return ' '.join(utt_as_list)

def shuffle_utterance(row):
    utt_as_list = row['text'].split()
    random.shuffle(utt_as_list)
    return ' '.join(utt_as_list)


In [273]:
######
# DEPRECATED
######

# MODIFYING FLUENT EXAMPLES
def inject_fake_disf(row,filler='euh'):
    utt_as_list = row['text'].split()
    number = int(max(1,int(len(utt_as_list) / 5))) 
    for x in range(0, number): # this line will ensure 4 element insertion
        insert_location = random.randrange(2, len(utt_as_list)-2)
        utt_as_list.insert(insert_location,filler)
        if random.random()> 0.5: 
            utt_as_list.insert(insert_location+1,'#')
    return ' '.join(utt_as_list)

In [274]:
DISFLUENT_DF['moved_disf'] = DISFLUENT_DF.apply(move_disfluencies,axis=1)
DISFLUENT_DF['removed_disf'] = DISFLUENT_DF.apply(remove_disfluencies,axis=1)
DISFLUENT_DF['replaced_disf'] = DISFLUENT_DF.apply(replace_disfluencies,axis=1)
DISFLUENT_DF['shuffled'] = DISFLUENT_DF.apply(shuffle_utterance,axis=1)

In [275]:
DISFLUENT_DF

,text,moved_disf,removed_disf,replaced_disf,shuffled
0,c' était un bar avec euh un un écran ou #,c' était un euh bar avec un un écran ou #,c' était un bar avec un un écran ou #,c' était un bar avec de un un écran ou #,un un avec # ou bar euh était écran c' un
1,je crois j' étais euh je me rappelle plus si j...,je crois j' étais je me euh rappelle plus si j...,je crois j' étais je me rappelle plus si j' ét...,je crois j' étais a je me rappelle plus si j' ...,étais plus première ou si j' année étais je au...
2,on a appelé euh des # * les parents d' amis #,on a appelé des # * euh les parents d' amis #,on a appelé des # * les parents d' amis #,on a appelé étais des # * les parents d' amis #,euh * appelé les on # a parents des d' amis #
3,mais jetés comme des euh # comme des miséreux ...,mais jetés comme euh des # comme des miséreux ...,mais jetés comme des # comme des miséreux sur ...,mais jetés comme des et # comme des miséreux s...,# mais des jetés bouffe euh la comme sur comme...
4,on en pouvait plus de euh d' attendre *,on en pouvait plus euh de d' attendre *,on en pouvait plus de d' attendre *,on en pouvait plus de pour d' attendre *,d' * euh pouvait en de plus on attendre
...,...,...,...,...,...
195,le fait d' avoir la ville et puis le fait d' a...,le fait d' avoir la ville et puis le fait d' a...,le fait d' avoir la ville et puis le fait d' a...,le fait d' avoir la ville et puis le fait d' a...,ville nature des puis le de de juste et de fai...
196,ah pff ah moi j' ai pas envie de repartir euh ...,ah pff ah moi j' euh ai pas envie de repartir ...,ah pff ah moi j' ai pas envie de repartir # de...,ah pff ah moi j' ai pas envie de repartir pas ...,la pas euh de de de de repartir région de j' a...
197,et qu' ils aient pas euh # euh # un terrain eu...,et qu' ils aient euh pas # euh # un euh euh te...,et qu' ils aient pas # # un terrain dans leque...,et qu' ils aient pas est # San-Francisco # un ...,tu partir vois dans lequel pas euh aient euh e...
198,mais euh # moi j' avais un j' avais un voisin ...,mais # moi j' avais un j' avais un euh voisin ...,mais # moi j' avais un j' avais un voisin qui ...,mais c' # moi j' avais un j' avais un voisin q...,à de euh un j' moi mais moi qui chez habitait ...


In [276]:
result = []
cnt=0
for i,row in DISFLUENT_DF.iterrows():
    item_shuffled = {"sentence_good":row['text'],"sentence_bad":row['shuffled'], "field": "disfluencies", 
                     "linguistics_term": "shuffled"+suffix, "UID": "shuffled"+suffix, "simple_LM_method": True, 
                     "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": True, 
                     "pair_id": str(cnt)}
    result.append(item_shuffled)
    cnt +=1
    
with open('shuffled' + suffix + '.json', 'w', encoding='utf-8') as outfile:
    for entry in result:
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')

In [277]:
result = []
cnt=0
for i,row in DISFLUENT_DF.iterrows():
    item_moved = {"sentence_good":row['text'], "sentence_bad":row['moved_disf'], "field": "disfluencies", 
                 "linguistics_term": "moved_filler"+suffix, "UID": "moved_filler"+suffix, "simple_LM_method": True,
                 "one_prefix_method": False, "two_prefix_method": False, 
                 "lexically_identical": True, "pair_id": str(cnt)}
    result.append(item_moved)
    cnt +=1
    
with open('moved_filler' + suffix + '.json', 'w', encoding='utf-8') as outfile:
    for entry in result:
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')

In [278]:
result = []
cnt=0
for i,row in DISFLUENT_DF.iterrows():
    item_removed = {"sentence_good":row['text'],"sentence_bad":row['removed_disf'], "field": "disfluencies", 
                    "linguistics_term": "removed_filler"+suffix, "UID": "removed_filler"+suffix, "simple_LM_method": True,
                    "one_prefix_method": False, "two_prefix_method": False, 
                    "lexically_identical": False, "pair_id": str(cnt)}
    result.append(item_removed)
    cnt +=1

    
with open('removed_filler' + suffix + '.json', 'w', encoding='utf-8') as outfile:
    for entry in result:
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')

In [279]:
result = []
cnt=0
for i,row in DISFLUENT_DF.iterrows():
    item_removed = {"sentence_good":row['text'],"sentence_bad":row['replaced_disf'], "field": "disfluencies", 
                    "linguistics_term": "replaced_filler"+suffix, "UID": "replaced_filler"+suffix, "simple_LM_method": True,
                    "one_prefix_method": False, "two_prefix_method": False, 
                    "lexically_identical": False, "pair_id": str(cnt)}
    result.append(item_removed)
    cnt +=1

    
with open('replaced_filler' + suffix + '.json', 'w', encoding='utf-8') as outfile:
    for entry in result:
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')